In [139]:
import pandas as pd

In [140]:
import glob
g = glob.glob('data/processed_games/[0-9]*.csv')
# df = pd.concat([pd.read_csv(n) for n in g])

In [141]:
def make_bigrams(g):
    se = g.court_location.shift(1)+g.court_location
    se = se.dropna()
    return se

In [142]:
# check out the number when we take in account player and basketball position
def get_rel_tobball(g):
    ball = g[g.team_id == -1]
    try:
        assert len(ball.index) == 1
        se = [ball.court_location.iloc[0]]*len(g.index)
    except:
        se = None
    g['ball_loc'] = se
    return g

# team_positions = grouped.apply(lambda x: get_rel_tobball(i, x, grouped))
# print(grouped.groups.keys())
# df_with_ball_loc['player_ball'] = df_with_ball_loc['ball_loc']+df_with_ball_loc['court_location']

def make_bigrams_player_ball(g):
    g['player_ball'] = g.player_ball.apply(lambda x: (x, ) if x is not None else None)
    g['bigram_player_ball'] = g.player_ball.shift(1)+g.player_ball
    return g

def get_counts(g):
    i, _g = g
    vc = _g['bigram_player_ball'].value_counts()
    index = pd.MultiIndex.from_tuples([i+(ind,) for ind in vc.index])
    d = pd.DataFrame(vc.values, index=index)
    return d


In [143]:
bigram_cts = None
for n in g:
    print (n)
    df = pd.read_csv(n)
    df_sorted = df.sort_values('game_clock')
    df_sorted['court_location'] = df_sorted['court_location'].apply(lambda x: (x,) if x is not None else None)

    grouped = df_sorted.groupby(['game_clock'])
    df_sorted = grouped.apply(get_rel_tobball)
#     print(df_sorted['court_location'])
    df_sorted['player_ball'] = df_sorted['ball_loc']+df_sorted['court_location']
    df_sorted = df_sorted.groupby(['player_id', 'event_id']).apply(make_bigrams_player_ball)

    if bigram_cts is None:
        df_sorted = df_sorted[['player_id', 'GAME_ID',
                                'bigram_player_ball']].dropna()
        df_sorted['bigram_player_ball_0'], df_sorted['bigram_player_ball_1'] = zip(*df_sorted['bigram_player_ball'].tolist())
        # need to split into indices since pandas can't handle nested tuples
        bigram_cts = df_sorted.groupby(['player_id', 'GAME_ID', 'bigram_player_ball_0', 'bigram_player_ball_1']).count()        
    else:
        df_sorted = df_sorted[['player_id', 'GAME_ID',
                                'bigram_player_ball']].dropna()
        df_sorted['bigram_player_ball_0'], df_sorted['bigram_player_ball_1'] = zip(*df_sorted['bigram_player_ball'].tolist())
        _cts = df_sorted.groupby(['player_id', 'GAME_ID', 'bigram_player_ball_0', 'bigram_player_ball_1']).count()
        bigram_cts = bigram_cts.add(_cts, fill_value=0.0)
        print(len(bigram_cts.index))
        
    del df

    

data/processed_games/0021500001.csv
data/processed_games/0021500002.csv
12802
data/processed_games/0021500003.csv
19480
data/processed_games/0021500004.csv
26437
data/processed_games/0021500005.csv
32852
data/processed_games/0021500007.csv
39237
data/processed_games/0021500009.csv
45673
data/processed_games/0021500010.csv
51679
data/processed_games/0021500011.csv
58424
data/processed_games/0021500012.csv
65376
data/processed_games/0021500013.csv
71968
data/processed_games/0021500015.csv
78796
data/processed_games/0021500016.csv
85518
data/processed_games/0021500017.csv
92139
data/processed_games/0021500018.csv
98441
data/processed_games/0021500019.csv
105048
data/processed_games/0021500020.csv
111955
data/processed_games/0021500021.csv
118360
data/processed_games/0021500022.csv
123769
data/processed_games/0021500023.csv
130276
data/processed_games/0021500024.csv
136973
data/processed_games/0021500025.csv
143132
data/processed_games/0021500027.csv
150012
data/processed_games/0021500028.

ValueError: setting an array element with a sequence.

In [ ]:
bigram_cts.to_csv("data/bigram_cts.csv")

In [9]:
df = df[['player_id', 'event_id', 'court_location', 'game_clock']]

In [10]:
df_sorted = df.sort_values('game_clock')

In [11]:
del df

In [12]:
df_sorted['court_location'] = df_sorted['court_location'].apply(lambda x: (x,))
# df_sorted['court_location'].value_counts()
# df_sorted['court_location']

In [14]:
def make_bigrams(g):
    se = g.court_location.shift(1)+g.court_location
    return se

df_sorted['bigrams'] = df_sorted.groupby(
          ['player_id', 'event_id']).apply(make_bigrams).reset_index(['event_id', 'player_id'])['court_location']

KeyboardInterrupt: 

In [ ]:
def make_trigrams(g):
    se = g.court_location.shift(2)+g.court_location.shift(1)+g.court_location
    return se

df_trigrams = df_bigrams.groupby(['player_id', 'event_id']).apply(make_trigrams, axis = 1)


In [ ]:
# sanity check
# for i, g in df_bigrams.groupby(['player_id', 'event_id']):
#     print (g[['court_location', 'bigram']])

In [ ]:
df_trigrams['trigram'].value_counts()

In [36]:
def make_team_tuples(g, axis = None):
#     print(g)
#     g['bigram'] = g.court_location.shift(1)+g.court_location
    
    if(len(g.index) == 5):
        return g.court_location.sum()
    return None

team_positions = df_sorted.groupby(['event_id','game_clock', 'team_id']).apply(make_bigrams, axis = 1)


In [73]:
# vc = team_positions.value_counts()
# vc[vc>1]

In [120]:
# test
# df_with_ball_loc[df_with_ball_loc.player_id == -1][['ball_loc', 'player_id', 'court_location']]
# df_with_ball_loc[df_with_ball_loc.player_id != -1][['ball_loc', 'player_id', 'court_location']]

In [123]:
df_with_ball_loc['player_ball'] = df_with_ball_loc['ball_loc']+df_with_ball_loc['court_location']

In [126]:
# df_with_ball_loc.player_ball.value_counts()

In [152]:
def make_bigrams_player_ball(g):
    g['player_ball'] = g.player_ball.apply(lambda x: (x, ) if x is not None else None)
    g['bigram_player_ball'] = g.player_ball.shift(1)+g.player_ball
    return g

df_bigrams_with_ball = df_with_ball_loc.groupby(['player_id', 'event_id']).apply(make_bigrams_player_ball)


In [154]:
vc = df_bigrams_with_ball['bigram_player_ball'].value_counts()
vc

((9.0, 9.0), (9.0, 9.0))        720
((12.0, 12.0), (12.0, 12.0))    617
((20.0, 20.0), (20.0, 20.0))    297
((16.0, 16.0), (16.0, 16.0))    266
((9.0, 12.0), (9.0, 12.0))      219
((12.0, 2.0), (12.0, 2.0))      154
((20.0, 12.0), (20.0, 12.0))    140
((9.0, 20.0), (9.0, 20.0))      138
((9.0, 16.0), (9.0, 16.0))      136
((12.0, 16.0), (12.0, 16.0))    111
((12.0, 11.0), (12.0, 11.0))    108
((16.0, 12.0), (16.0, 12.0))     99
((20.0, 2.0), (20.0, 2.0))       98
((12.0, 9.0), (12.0, 9.0))       98
((12.0, 20.0), (12.0, 20.0))     90
((9.0, 6.0), (9.0, 6.0))         87
((9.0, 2.0), (9.0, 2.0))         87
((9.0, 10.0), (9.0, 10.0))       86
((12.0, 6.0), (12.0, 6.0))       86
((12.0, 10.0), (12.0, 10.0))     75
((20.0, 16.0), (20.0, 16.0))     75
((16.0, 9.0), (16.0, 9.0))       73
((16.0, 6.0), (16.0, 6.0))       72
((12.0, 15.0), (12.0, 15.0))     69
((6.0, 6.0), (6.0, 6.0))         68
((20.0, 19.0), (20.0, 19.0))     62
((12.0, 19.0), (12.0, 19.0))     61
((20.0, 11.0), (20.0, 11.0))

In [255]:
def make_sparse_mat(df, dictionary, crit):
    gs = df.groupby(['GAME_ID', 'player_id'])
    mat = sp.dok_matrix((len(gs),len(dictionary.keys())), dtype=np.int8)
    unique_plays = 0
    for i, _gs in enumerate(gs):
        _, _g = _gs 
        player_game_stats = _g[crit].value_counts()
        for k, v in player_game_stats.items():
            mat[i, dictionary[k]] = v
#     vc = g.apply(lambda x: x[crit].value_counts())
    return mat.transpose().tocsr()
    # word->id

In [256]:
def make_dictionary(df, crit):
    d = {}
    gs = df.groupby(['GAME_ID', 'player_id'])
    unique_plays = 0
    for i, _g in gs:
        player_game_stats = _g[crit].value_counts()
        
        for k, v in player_game_stats.items():
            if k not in d.keys():
                d[k] = unique_plays
                unique_plays+=1
#     vc = g.apply(lambda x: x[crit].value_counts())
    return d
    # word->id

def make_id2word(dictionary):
    d = {}
    for k, v in dictionary.items():
        d[v] = k
    return d
    # id->word
    

In [257]:
# gs = df_bigrams_with_ball[['player_id', 'bigram_player_ball']].dropna()
# gs = gs.groupby(['player_id'])
# dictionary = {}
# for i, g in gs:
# #     transformed.append()
# #     print(g.bigram_player_ball.value_counts())
#     print(i)
dictionary = make_dictionary(df_bigrams_with_ball, 'bigram_player_ball')
make_sparse_mat(df_bigrams_with_ball, dictionary, 'bigram_player_ball')

<2351x19 sparse matrix of type '<class 'numpy.int8'>'
	with 6679 stored elements in Compressed Sparse Row format>

In [201]:
df_bigrams_with_ball.columns

Index(['Unnamed: 0', 'event_id', 'game_clock', 'home', 'player_id', 'radius',
       'shot_clock', 'team_id', 'visitor', 'x_loc', 'y_loc', 'court_location',
       'GAME_ID', 'EVENTNUM', 'EVENTMSGTYPE', 'EVENTMSGACTIONTYPE', 'PERIOD',
       'WCTIMESTRING', 'PCTIMESTRING', 'HOMEDESCRIPTION', 'NEUTRALDESCRIPTION',
       'VISITORDESCRIPTION', 'SCORE', 'SCOREMARGIN', 'PERSON1TYPE',
       'PLAYER1_ID', 'PLAYER1_NAME', 'PLAYER1_TEAM_ID', 'PLAYER1_TEAM_CITY',
       'PLAYER1_TEAM_NICKNAME', 'PLAYER1_TEAM_ABBREVIATION', 'PERSON2TYPE',
       'PLAYER2_ID', 'PLAYER2_NAME', 'PLAYER2_TEAM_ID', 'PLAYER2_TEAM_CITY',
       'PLAYER2_TEAM_NICKNAME', 'PLAYER2_TEAM_ABBREVIATION', 'PERSON3TYPE',
       'PLAYER3_ID', 'PLAYER3_NAME', 'PLAYER3_TEAM_ID', 'PLAYER3_TEAM_CITY',
       'PLAYER3_TEAM_NICKNAME', 'PLAYER3_TEAM_ABBREVIATION', 'ball_loc',
       'player_ball', 'bigram_player_ball'],
      dtype='object')